In [ ]:
# This notebook assumes the state machine in this directory has been instantiated:
# https://github.com/d-smith/workflow-patterns-aws/tree/master/error-handling/reoffer

In [ ]:
import boto3

client = boto3.client('stepfunctions')

In [ ]:
result = client.list_state_machines()
print(result)

In [ ]:
sma = result['stateMachines'][0]['stateMachineArn']
print(sma)

In [ ]:
# Instantiate an instance
result = client.start_execution(
    stateMachineArn=sma,
    input='{"foo": true}'
)

print(result);

In [ ]:
response = client.list_executions(
    stateMachineArn=sma,
    statusFilter='RUNNING'
)

print(response)

if len(response['executions']) > 0:
    firstExecution = response['executions'][0]['executionArn']
    print(firstExecution)
else:
    print('No executions!')

In [ ]:
paginator = client.get_paginator('get_execution_history')
for response in paginator.paginate(executionArn=firstExecution):
    print(response)

In [ ]:
# Look at the last event - is this the manual activity?
response = client.get_execution_history(
    executionArn=firstExecution,
    maxResults=1,
    reverseOrder=True
)


event = response['events'][0]
eventType = event['type']
activity = event['activityScheduledEventDetails']['resource']
print(event)


if eventType == 'ActivityScheduled' and activity.endswith('manual-reoffer-dev'):
    print('Manual activity is ready to claim')


In [ ]:
import random
import string
N=5
worker = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))

# Claim manual activity
response = client.get_activity_task(
    activityArn=activity,
    workerName='worker'
)

print(response)

In [ ]:
# Signal activity complete - note if you delay the execution will time out.
response = client.send_task_success(
    taskToken=response['taskToken'],
    output='{"m1":"all ok"}',
)

print(response)

In [ ]:
# Look at the last event - is this the manual activity?
response = client.get_execution_history(
    executionArn=firstExecution,
    maxResults=1,
    reverseOrder=True
)

print(response)

if response['events'][0]['type'] == 'ExecutionSucceeded':
    print('Execution succeeded')
else:
    print('Huh?')